In [1]:
from tree import Mbtree

mbtree = Mbtree.load("../data/abtree_root")
mbtree.calc_score_by_ab(mbtree.root, use_tt=True, debug=True)

ミニマックス法で計算したか False
計算したノードの数 999
枝狩りしたノードの数 548947
合計 549946
割合 0.2%


In [2]:
from ai import dprint, ai_by_score
from marubatsu import Marubatsu
from copy import deepcopy

@ai_by_score
def ai_abs_tt(mb, debug=False, shortest_victory=False):            
    count = 0
    def ab_search(mborig, tt, alpha=float("-inf"), beta=float("inf")):
        nonlocal count
        count += 1
        if mborig.status == Marubatsu.CIRCLE:
            return (11 - mborig.move_count) / 2 if shortest_victory else 1
        elif mborig.status == Marubatsu.CROSS:
            return (mborig.move_count - 10) / 2 if shortest_victory else -1
        elif mborig.status == Marubatsu.DRAW:
            return 0
        
        boardtxt = mborig.board_to_str()
        if boardtxt in tt:
            return tt[boardtxt]

        legal_moves = mborig.calc_legal_moves()
        if mborig.turn == Marubatsu.CIRCLE:
            for x, y in legal_moves:
                mb = deepcopy(mborig)
                mb.move(x, y)
                score = ab_search(mb, tt, alpha, beta)
                if score >= beta:
                    alpha = score
                    break
                alpha = max(alpha, score)
            score = alpha
        else:
            for x, y in legal_moves:
                mb = deepcopy(mborig)
                mb.move(x, y)
                score = ab_search(mb, tt, alpha, beta)
                if score <= alpha:
                    beta = score
                    break
                beta = min(beta, score)   
            score = beta
            
        from util import calc_same_boardtexts

        boardtxtlist = calc_same_boardtexts(mborig)
        for boardtxt in boardtxtlist:
            tt[boardtxt] = score
        return score
                
    score = ab_search(mb, {})
    dprint(debug, "count =", count)
    if mb.turn == Marubatsu.CIRCLE:
        score *= -1
    return score

In [3]:
from util import Check_solved

result, incorrectlist = Check_solved.is_strongly_solved(ai_abs_tt)
print(result)

100%|██████████| 431/431 [00:03<00:00, 132.59it/s]

413/431 95.82%
False


In [4]:
for mb, candidate, bestmoves in incorrectlist:
    print(mb)
    print("candidate", candidate)
    print("bestmoves", bestmoves)

Turn x
O..
...
...

candidate {(0, 1), (1, 2), (2, 1), (1, 1), (2, 0), (0, 2), (2, 2), (1, 0)}
bestmoves {(1, 1)}
Turn x
oxO
...
...

candidate {(0, 1), (1, 2), (2, 1), (1, 1), (0, 2), (2, 2)}
bestmoves {(1, 1)}
Turn o
ox.
o.X
...

candidate {(2, 2), (1, 1), (2, 0), (0, 2)}
bestmoves {(1, 1), (2, 2), (0, 2)}
Turn o
ox.
.o.
..X

candidate {(0, 1), (0, 2), (2, 1)}
bestmoves {(0, 1), (0, 2)}
Turn o
ox.
..o
..X

candidate {(0, 1), (1, 1), (0, 2)}
bestmoves {(0, 1)}
Turn o
ox.
...
o.X

candidate {(0, 1), (1, 2), (2, 1), (1, 1), (2, 0)}
bestmoves {(0, 1), (1, 1), (2, 0)}
Turn x
o.x
.O.
...

candidate {(0, 1), (1, 2), (2, 1), (0, 2), (2, 2), (1, 0)}
bestmoves {(2, 2)}
Turn o
o.x
..o
..X

candidate {(0, 1), (1, 1), (0, 2)}
bestmoves {(0, 1)}
Turn o
o.x
...
o.X

candidate {(0, 1), (2, 1)}
bestmoves {(0, 1)}
Turn x
o.O
.x.
...

candidate {(0, 1), (1, 2), (2, 1), (0, 2), (2, 2), (1, 0)}
bestmoves {(1, 0)}
Turn o
o..
..X
...

candidate {(0, 1), (1, 1), (2, 0), (0, 2)}
bestmoves {(1, 1), (2, 0), (0

In [5]:
node = mbtree.root.children_by_move[(0, 0)].children_by_move[(1, 0)].children_by_move[(0, 1)].children_by_move[(2, 1)]
print(node.mb)

Turn o
ox.
o.X
...



In [6]:
ai_abs_tt(node.mb, analyze=True)

{'candidate': [(2, 0), (1, 1), (0, 2), (2, 2)],
 'score_by_move': {(2, 0): 1, (1, 1): 1, (0, 2): 1, (1, 2): 0, (2, 2): 1}}

In [7]:
from tree import Mbtree_Anim

mbtree.calc_score_by_ab(node, use_tt=True)
Mbtree_Anim(mbtree, isscore=True)

In [8]:
@ai_by_score
def ai_abs_tt(mb, debug=False, shortest_victory=False):            
    count = 0
    def ab_search(mborig, tt, alpha=float("-inf"), beta=float("inf")):
        nonlocal count
        count += 1
        if mborig.status == Marubatsu.CIRCLE:
            return (11 - mborig.move_count) / 2 if shortest_victory else 1
        elif mborig.status == Marubatsu.CROSS:
            return (mborig.move_count - 10) / 2 if shortest_victory else -1
        elif mborig.status == Marubatsu.DRAW:
            return 0
        
        boardtxt = mborig.board_to_str()
        key = (boardtxt, alpha, beta)
        if key in tt:
            return tt[key]

        legal_moves = mborig.calc_legal_moves()
        if mborig.turn == Marubatsu.CIRCLE:
            for x, y in legal_moves:
                mb = deepcopy(mborig)
                mb.move(x, y)
                score = ab_search(mb, tt, alpha, beta)
                if score >= beta:
                    alpha = score
                    break
                alpha = max(alpha, score)
            score = alpha
        else:
            for x, y in legal_moves:
                mb = deepcopy(mborig)
                mb.move(x, y)
                score = ab_search(mb, tt, alpha, beta)
                if score <= alpha:
                    beta = score
                    break
                beta = min(beta, score)   
            score = beta
            
        from util import calc_same_boardtexts

        boardtxtlist = calc_same_boardtexts(mborig)
        _, alpha, beta = key
        for boardtxt in boardtxtlist:
            tt[(boardtxt, alpha, beta)] = score
        return score
                
    score = ab_search(mb, {})
    dprint(debug, "count =", count)
    if mb.turn == Marubatsu.CIRCLE:
        score *= -1
    return score

In [9]:
result, incorrectlist = Check_solved.is_strongly_solved(ai_abs_tt)
print(result)

100%|██████████| 431/431 [00:03<00:00, 121.32it/s]

431/431 100.00%
True


In [10]:
result, incorrectlist = Check_solved.is_strongly_solved(ai_abs_tt, 
                                                        params={"shortest_victory": True})
print(result)

100%|██████████| 431/431 [00:04<00:00, 99.99it/s] 

431/431 100.00%
True


In [11]:
def calc_score_by_ab(self, abroot, debug=False, minimax=False, init_ab=False, use_tt=False):
    def assign_pruned_index(node, index):
        node.pruned_index = index
        self.num_pruned += 1
        for childnode in node.children:
            assign_pruned_index(childnode, index)
        
    def calc_ab_score(node, tt, alpha=float("-inf"), beta=float("inf")):
        if minimax:
            alpha = float("-inf")
            beta = float("inf")
        if use_tt:
            boardtxt = node.mb.board_to_str()
            key = (boardtxt, alpha, beta)
            if key in tt:
                node.score = tt[key]
                if node.mb.turn == Marubatsu.CIRCLE:
                    alpha = node.score
                else:
                    beta = node.score
                self.nodelist_by_score.append(node)
                self.num_calculated += 1     
                for childnode in node.children:
                    assign_pruned_index(childnode, len(self.nodelist_by_score) - 1)
                self.ablist_by_score.append((alpha, beta, None, "tt",
                                             self.num_calculated, self.num_pruned))
                node.score_index = len(self.nodelist_by_score) - 1  
                return node.score
                
        self.nodelist_by_score.append(node)
        self.ablist_by_score.append((alpha, beta, None, "start",
                                    self.num_calculated, self.num_pruned))
        if node.mb.status != Marubatsu.PLAYING:
            self.calc_score_of_node(node)
            if node.mb.turn == Marubatsu.CIRCLE:
                alpha = node.score
            else:
                beta = node.score
        else:
            if node.mb.turn == Marubatsu.CIRCLE:
                for childnode in node.children:
                    score = calc_ab_score(childnode, tt, alpha, beta)
                    self.nodelist_by_score.append(node)
                    self.ablist_by_score.append((alpha, beta, score, "score",
                                                self.num_calculated, self.num_pruned))
                    if score > alpha:
                        alpha = score
                    self.nodelist_by_score.append(node)
                    if score >= beta:
                        index = node.children.index(childnode)
                        for prunednode in node.children[index + 1:]:
                            assign_pruned_index(prunednode, len(self.nodelist_by_score) - 1)
                    self.ablist_by_score.append((alpha, beta, None, "update",
                                                self.num_calculated, self.num_pruned))
                    if score >= beta:
                        break
                node.score = alpha
            else:
                for childnode in node.children:
                    score = calc_ab_score(childnode, tt, alpha, beta)
                    self.nodelist_by_score.append(node)
                    self.ablist_by_score.append((alpha, beta, score, "score",
                                                self.num_calculated, self.num_pruned))
                    if score < beta:
                        beta = score
                    self.nodelist_by_score.append(node)
                    if score <= alpha:
                        index = node.children.index(childnode)
                        for prunednode in node.children[index + 1:]:
                            assign_pruned_index(prunednode, len(self.nodelist_by_score) - 1)
                    self.ablist_by_score.append((alpha, beta, None, "update",
                                                self.num_calculated, self.num_pruned))
                    if score <= alpha:
                        break
                node.score = beta

        self.nodelist_by_score.append(node)
        self.num_calculated += 1     
        self.ablist_by_score.append((alpha, beta, None, "end",
                                    self.num_calculated, self.num_pruned))
        node.score_index = len(self.nodelist_by_score) - 1  
        if use_tt:
            from util import calc_same_boardtexts   
            
            boardtxtlist = calc_same_boardtexts(node.mb)
            _, alpha, beta = key
            for boardtxt in boardtxtlist:
                tt[(boardtxt, alpha, beta)] = node.score            
        return node.score

    from ai import dprint       
    for node in self.nodelist:
        node.score_index = float("inf")
        node.pruned_index = float("inf")
    self.nodelist_by_score = []
    self.ablist_by_score = []
    self.num_calculated = 0
    self.num_pruned = 0
    if init_ab:
        alpha = -2 if self.shortest_victory else -1
        beta = 3 if self.shortest_victory else 1
    else:
        alpha = float("-inf")
        beta = float("inf")
    calc_ab_score(abroot, {}, alpha, beta)
    total_nodenum = self.num_pruned + self.num_calculated
    ratio = self.num_calculated / total_nodenum * 100
    dprint(debug, "ミニマックス法で計算したか", minimax)    
    dprint(debug, "計算したノードの数",  self.num_calculated)
    dprint(debug, "枝狩りしたノードの数",  self.num_pruned)
    dprint(debug, "合計",  total_nodenum)
    dprint(debug, f"割合 {ratio:.1f}%")
    
Mbtree.calc_score_by_ab = calc_score_by_ab

In [12]:
mbtree.calc_score_by_ab(node, use_tt=True)
Mbtree_Anim(mbtree, isscore=True)

In [ ]:
print("ミニマックス法で置換表を利用しない")
mbtree.calc_score_by_ab(mbtree.root, minimax=True, debug=True)
print()
print("ミニマックス法で置換表を利用する")
mbtree.calc_score_by_ab(mbtree.root, minimax=True, use_tt=True, debug=True)
print()
print("αβ 法で置換表を利用しない")
mbtree.calc_score_by_ab(mbtree.root, debug=True)
print()
print("αβ 法で置換表を利用する")
mbtree.calc_score_by_ab(mbtree.root, use_tt=True, debug=True)

ミニマックス法で置換表を利用しない
ミニマックス法で計算したか True
計算したノードの数 549946
枝狩りしたノードの数 0
合計 549946
割合 100.0%

ミニマックス法で置換表を利用する
ミニマックス法で計算したか True
計算したノードの数 2271
枝狩りしたノードの数 547675
合計 549946
割合 0.4%

αβ 法で置換表を利用しない
ミニマックス法で計算したか False
計算したノードの数 18297
枝狩りしたノードの数 531649
合計 549946
割合 3.3%

αβ 法で置換表を利用する
ミニマックス法で計算したか False
計算したノードの数 1718
枝狩りしたノードの数 548228
合計 549946
割合 0.3%


In [14]:
mbtree_sv = Mbtree.load("../data/abtree_sv_root")

print("ミニマックス法で置換表を利用しない")
mbtree_sv.calc_score_by_ab(mbtree_sv.root, minimax=True, debug=True)
print()
print("ミニマックス法で置換表を利用する")
mbtree_sv.calc_score_by_ab(mbtree_sv.root, minimax=True, use_tt=True, debug=True)
print()
print("αβ 法で置換表を利用しない")
mbtree_sv.calc_score_by_ab(mbtree_sv.root, debug=True)
print()
print("αβ 法で置換表を利用する")
mbtree_sv.calc_score_by_ab(mbtree_sv.root, use_tt=True, debug=True)

ミニマックス法で置換表を利用しない
ミニマックス法で計算したか True
計算したノードの数 549946
枝狩りしたノードの数 0
合計 549946
割合 100.0%

ミニマックス法で置換表を利用する
ミニマックス法で計算したか True
計算したノードの数 2271
枝狩りしたノードの数 547675
合計 549946
割合 0.4%

αβ 法で置換表を利用しない
ミニマックス法で計算したか False
計算したノードの数 20866
枝狩りしたノードの数 529080
合計 549946
割合 3.8%

αβ 法で置換表を利用する
ミニマックス法で計算したか False
計算したノードの数 2583
枝狩りしたノードの数 547363
合計 549946
割合 0.5%
